# Investigation of double derivative convergence fail

It is observe that `DDX(DDX(f))` fails to converge, whereas `D2DX2(f)` doesn't.

Let's call the FD operator $D$

$$
D[f_i] = \frac{-f_{i-1} + f_{i+1}}{hx}
$$

Thus

$$
D[D[f_i]] = D\left[\frac{-f_{i-1} + f_{i+1}}{hx}\right]
= \frac{-D[f_{i-1}] + D[f_{i+1}]}{hx}
= \frac{-\frac{-f_{i-2} + f_{i}}{hx} + \frac{-f_{i} + f_{i+2}}{hx}}{hx}
= \frac{f_{i-2} - 2f_{i} + f_{i+2}}{hx^2}
$$

in other words, we see that we need information from $f_{i-2}$. Assuming now that $i$ denotes the first inner point. $f_{i-2}$ is in principle known (as this is point is the second innermost point diametrically opposite of $i$), but after calculating the first $D[f]$, part of this information is lost when we are setting the ghost point for the innermost points.